In [1]:
import cv2
import numpy as np
import scipy.spatial.distance as distance
from sklearn.preprocessing import normalize
import pandas as pd
import time

In [2]:
f = open("data/data_idx.txt", "r")
clusters = [int(x) for x in f.readline().split()]
clusters = np.asarray(clusters)
image_cluster = pd.DataFrame(clusters)
clusters, requests, Nc = np.unique(clusters, return_index=True,
                                      return_counts=True)
image_cluster

,0
0,1
1,1
2,1
3,2
4,2
...,...
1486,499
1487,499
1488,500
1489,500


In [3]:
f = open("data/bow.txt", "r")
matrix = []
for line in f:
    vector = [int(x) for x in line.split()]
    matrix.append(vector)
bow_matrix = np.asmatrix(matrix)
bow = pd.DataFrame(bow_matrix)
bow

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,1,0,5,0,1,0,1,1,0,0,...,1,0,1,0,0,0,0,0,1,0
1,5,0,4,2,2,1,2,11,7,1,...,4,11,1,0,4,0,0,3,0,0
2,1,0,0,0,1,0,0,0,2,0,...,2,0,0,0,0,0,1,0,0,0
3,18,2,10,4,12,8,7,1,18,7,...,4,9,4,1,19,6,2,0,5,8
4,13,1,7,2,11,11,4,0,13,9,...,1,3,2,1,8,3,3,3,2,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1486,11,3,7,13,1,2,8,1,9,4,...,1,3,6,9,2,8,2,1,3,1
1487,7,5,6,9,1,0,6,0,1,10,...,5,1,13,1,5,0,2,0,4,3
1488,0,0,0,9,0,1,1,0,2,2,...,0,1,5,7,1,1,0,3,1,3
1489,0,0,0,2,3,0,0,0,12,0,...,0,0,0,2,0,0,0,0,0,0


There are 1491 images in this database, and 8 clusters. (1024 / 128) Therefore, we retrieve the centroids of these 8 clusters.

In [5]:
f = open("data/vlad.txt", "r")
matrix = []
for line in f:
    vector = [float(x) for x in line.split()]
    matrix.append(vector)
vlad_matrix = np.asmatrix(matrix)
vlad = pd.DataFrame(vlad_matrix)
vlad

,0,1,2,3,4,5,6,7,8,9,...,8182,8183,8184,8185,8186,8187,8188,8189,8190,8191
0,-21.893009,174.355240,58.976070,-83.114029,-337.857391,41.943695,-39.480526,-40.694977,-40.251991,34.622242,...,27.620071,356.911011,-62.078690,-111.560577,-75.615868,-18.480951,105.407257,-9.619627,11.096497,471.302551
1,367.745422,284.111694,3.977940,-510.281250,-752.672668,188.712845,84.949310,84.555618,16.414883,352.573273,...,144.943237,641.761475,180.662415,133.883240,107.503464,27.938805,117.888222,-82.369835,-141.015015,326.010864
2,-5.789301,-3.664477,-6.902393,9.288597,-7.385735,-11.005630,7.351947,15.030502,-16.325199,-4.537776,...,-36.691437,-53.588379,-25.014994,85.083702,-8.355402,-9.758276,-11.160524,66.072449,-48.267334,-69.799500
3,18.113567,56.105122,-32.314880,432.852997,-124.515404,57.729736,-37.106518,-111.535889,-108.609436,100.186775,...,-1074.199585,-1.593262,-121.524612,146.929062,-254.439224,113.460495,-126.618301,-963.462830,-572.357849,-286.984070
4,9.267487,80.388092,79.440231,188.215118,-64.643356,-66.140762,-46.201309,-69.237442,-215.129913,-15.444384,...,-161.665558,-420.798462,271.545990,420.532898,-195.500992,-108.437737,4.394178,-662.806946,-776.586853,113.315140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1486,-160.990128,-116.875122,-188.436279,455.450165,57.013000,-262.320953,-140.939041,-118.261368,-311.536163,-25.653223,...,335.403320,-352.208862,285.688843,-243.263504,-19.624615,157.765533,143.419159,-337.496552,111.952820,-12.089615
1487,677.173462,136.532700,-94.117706,-314.737183,241.897583,-531.511719,-49.972965,576.776062,350.407104,60.062473,...,-346.065643,-111.501831,92.808853,31.067188,-41.781387,-16.168026,9.703335,-163.076218,-372.908508,-166.693619
1488,-70.618896,-102.506775,-183.874619,-68.932045,976.840942,-299.410400,93.692078,-208.773331,-133.739517,116.742249,...,-335.379974,125.911011,177.921234,-96.560562,176.384125,26.519047,-33.592747,-296.619598,-205.903503,203.302628
1489,7.842796,23.342093,11.390427,161.154419,-55.542938,-77.022522,-26.592211,-18.877991,78.699219,11.848898,...,59.385704,183.264526,120.981239,-27.895378,-24.194252,-13.447845,-13.950655,44.090561,32.165833,134.500626


VLAD used 64 clusters. (8192 / 128)

In [7]:
def getListOfSimilarImages(request, database):
    distances = distance.cdist(request[None, :], 
                               np.asmatrix(database), 
                               metric='euclidean').transpose()
    distances = pd.DataFrame(distances)
    distances = distances.sort_values(distances.columns[0])
    return distances.index

In [8]:
distances = getListOfSimilarImages(bow.values[2], bow.values)
distances

Int64Index([   2,  698,  697,  696, 1194,  763,  764,  748,   31,  917,
            ...
             114,   51, 1361, 1039, 1011, 1067, 1073, 1071,  862, 1075],
           dtype='int64', length=1491)

In [9]:
def getAllSimilar(database, indexes):
    distances = []
    for i in range(500):
        distances.append(getListOfSimilarImages(database[indexes[i]],
                                           database).to_numpy())
    return np.array(distances)

In [87]:
def getAP(request, database, Nc, index):
    ap = 0
    indexes = np.array([i + index for i in range(Nc)])
    distances = getListOfSimilarImages(request, database).to_numpy()
    positions = np.where(np.in1d(distances, indexes))[0]
    for i in range(Nc):
        ap += (i + 1) / (positions[i] + 1)
    ap /= Nc
    return ap

In [88]:
getAP(bow.iloc[0], bow, Nc[0], 0)

0.35451960550385747

In [89]:
def getMAP(database, indexes, Ncs):
    MAP = 0
    for i in range(500):
        MAP += getAP(database[indexes[i]], database, Ncs[i], indexes[i])
    return MAP / 500

In [90]:
start_time = time.time()
bowMAP = getMAP(bow.values, requests, Nc)
elapsed_time = time.time() - start_time
print("MAP of bow is " + str(bowMAP) + " in " + str(elapsed_time) + 
      " seconds.")

MAP of bow is 0.6440300632260709 in 1.619802713394165 seconds.


In [91]:
start_time = time.time()
vladMAP = getMAP(vlad.values, requests, Nc)
elapsed_time = time.time() - start_time
print("MAP of vlad is " + str(vladMAP)+ " in " + str(elapsed_time) + 
      " seconds.")

MAP of vlad is 0.5668178186091257 in 3.794431686401367 seconds.


In [92]:
nbow  = normalize(bow.values, norm="l1")
nvlad = normalize(vlad.values, norm="l2")

In [93]:
nbowMAP = getMAP(nbow, requests, Nc)
print("MAP of bow normalized is " + str(nbowMAP))

MAP of bow normalized is 0.598430934118102


In [94]:
nvladMAP = getMAP(nvlad, requests, Nc)
print("MAP of vlad normalized is " + str(nvladMAP))

MAP of vlad normalized is 0.7341197402393242


In [84]:
def AP(y_true, d_pred):
    Nc = np.sum(y_true)
    ap = 0
    true = np.where(y_true)
    positions = np.where(np.in1d(d_pred, true))[0]
    for i in range(Nc):
        ap += (i + 1) / (positions[i] + 1)
    ap /= Nc
    return ap

In [85]:
def MAP(y_true, d_pred):
    MAP = 0
    for i in range(500):
        MAP += AP(y_true[i], d_pred[i])
    return MAP / 500

In [97]:
def getTruthMatrix(requests, results):
    true_vals = []
    for j in range(500):
        y_true = np.zeros(1491, dtype=bool)
        pred_v = np.zeros(1491, dtype=bool)
        if(j != 0):
            y_true[requests[j]:i+requests[j]] = True
        else:
            y_true[requests[j]:i+requests[j]+1] = True
        y_true.astype(bool)
        true_vals.append(y_true)
    return true_vals

In [99]:
### Testing new function
results = getAllSimilar(nbow, requests)
true_vals = getTruthMatrix(requests, results)
val = MAP(true_vals, results)
print("MAP for normalized BOW : " + str(val))

MAP for normalized BOW : 0.6843607977012095


In [101]:
results = getAllSimilar(nvlad, requests)
true_vals = getTruthMatrix(requests, results)
val = MAP(true_vals, results)
print("MAP for normalized VLAD : " + str(val))

MAP for normalized VLAD : 0.7954745633229371
